In [ ]:
import os
import pandas as pd
from scipy.integrate import cumulative_trapezoid
from datetime import datetime
import pytz

localtime = pytz.timezone('Europe/London')

In [ ]:
def process_files_and_save(folder_path, resample_period=None, output_folder="output"):
    filenames = os.listdir(folder_path)
    csv_filenames = [f for f in filenames if f.endswith('.csv')]
    if not csv_filenames:
        raise ValueError("No CSV files found in the folder.")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for fname in csv_filenames:
        df = pd.read_csv(os.path.join(folder_path, fname)).drop(columns=["Sync Byte 1", "Sync Byte 2", "Heartbeat"])
        df.index = pd.to_datetime(df.runner_timestamp, unit="s", origin="unix")
        df["delta_t"] = df.message_timestamp.diff().fillna(0)
        df["t"] = df.message_timestamp - df.message_timestamp.iloc[0]
        df["volume"] = cumulative_trapezoid(df["MFC flow rate"]/60, x=df.t, dx=df.delta_t.mean(), initial=0)
        if resample_period:
            df = df.resample(resample_period).mean(numeric_only=True)

        # Extract date and time information for constructing the new filename
        date_time = df.index[0].strftime("%Y-%m-%dT%Hh%M")
        new_filename = f"{fname[:3]}-{date_time}-10s.csv"
        new_filename = os.path.join(output_folder, new_filename.replace("_", "h"))

        # Save the processed DataFrame as a new CSV file
        df.to_csv(new_filename)

In [ ]:
folder_path = "C:\\Users\\Sebastiano Gadolini\\Documents\\Python Scripts\\MogRig_ProcessRawData\\"
resample_period = "10s"

process_files_and_save(folder_path, resample_period)